##  SETUP VIRTUAL MACHINES FOR MULTIPLE USERS
Use Azure Virtual Machine Scale Set (VMSS) and Azure Command Line Interface (Az CLI) to create N virtual machines. After setting up we invoke a setup script on each VM to setup the environment. 



## Azure Resource Setup